In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import gym
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Step 1: Generate Synthetic FANET Dataset
np.random.seed(42)
n_samples = 5000
data = pd.DataFrame({
    'node_speed': np.random.uniform(0, 20, n_samples),  # Speed in m/s
    'link_quality': np.random.uniform(0, 1, n_samples),  # 0 (poor) to 1 (excellent)
    'congestion_level': np.random.uniform(0, 1, n_samples),
    'energy': np.random.uniform(10, 100, n_samples),  # Battery level in %
    'hop_count': np.random.randint(1, 10, n_samples),
    'best_next_hop': np.random.randint(0, 5, n_samples)  # 5 possible next-hop options
})

data.to_csv('fanet_routing_data.csv', index=False)

# Step 2: Feature Engineering
features = ['node_speed', 'link_quality', 'congestion_level', 'energy', 'hop_count']
target = 'best_next_hop'

X = data[features]
y = data[target]

# Normalize Features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split Data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Step 3: Train GBDT Model
gbdt_model = lgb.LGBMClassifier()
gbdt_model.fit(X_train, y_train)

def gbdt_predict(state):
    state_scaled = scaler.transform([state])
    return gbdt_model.predict(state_scaled)[0]

# Step 4: Reinforcement Learning with DQN
class FANETEnv(gym.Env):
    def __init__(self):
        self.state_space = gym.spaces.Box(low=0, high=1, shape=(len(features),), dtype=np.float32)
        self.action_space = gym.spaces.Discrete(5)  # Assume 5 routing choices
        self.state = np.random.rand(len(features))

    def step(self, action):
        reward = np.random.rand()  # Simulated reward
        self.state = np.random.rand(len(features))
        return self.state, reward, False, {}

    def reset(self):
        self.state = np.random.rand(len(features))
        return self.state

env = FANETEnv()

# Define RL Model (Deep Q-Network)
n_inputs = len(features)
n_outputs = 5  # Number of routing options

model = tf.keras.Sequential([
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_outputs, activation='linear')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss='mse')

# Hybrid Routing Decision Function
def hybrid_routing(state):
    gbdt_action = gbdt_predict(state)
    if np.random.rand() < 0.8:
        return gbdt_action  # Use GBDT 80% of the time
    else:
        q_values = model.predict(np.array([state]))
        return np.argmax(q_values)

# Example Decision
state = np.random.rand(len(features))
chosen_action = hybrid_routing(state)
print("Selected Routing Action:", chosen_action)


